# Data Cleaning 

1. We noticed that scraped content after parsing to markdown had introduced a lot of whitespaces in the string
2. We use regex to remove all extra whitespaces to store the results in a new column "md_content_cleaned"
3. This script runs fairly quickly < 10min

In [6]:
import singlestoredb as s2
import re
from tqdm import tqdm

# Define regex pattern for cleaning text: collapse runs of spaces, tabs, and newlines
whitespace_pattern = re.compile(r'[ \t\r\n]+')

def clean_text(text):
    """Collapse all runs of whitespace (spaces, tabs, newlines) and trim."""
    return whitespace_pattern.sub(' ', text).strip()

def process_rows():
    # Connect to the SingleStore database
    conn = s2.connect(
        database='knowlagent'
    )

    try:
        with conn.cursor() as cursor:
            create_cleaned_md_column = """
            ALTER TABLE {TABLE_NAME} ADD COLUMN md_content_cleaned LONGTEXT NULL;
            """
            cursor.execute(create_cleaned_md_column)
            
            # Optionally, find the initial number of rows to process for a global progress bar.
            count_query = """
                SELECT COUNT(*)
                FROM s2docs
                WHERE (md_content IS NOT NULL OR length(md_content_cleaned)>10)
                AND (md_content_cleaned IS NULL OR md_content_cleaned = '');
            """
            cursor.execute(count_query)
            total_rows = cursor.fetchone()[0]
            pbar = tqdm(total=total_rows, desc="Processing rows", unit="row")

            # Query to select rows that need processing, batched by a LIMIT clause.
            select_sql = """
                SELECT id, md_content
                FROM s2docs
                WHERE (md_content IS NOT NULL OR length(md_content_cleaned)>10)
                AND (md_content_cleaned IS NULL OR md_content_cleaned = '')
                LIMIT 100;
            """
            # Prepared update statement for each row.
            update_sql = "UPDATE s2docs SET md_content_cleaned = %s WHERE id = %s"

            while True:
                cursor.execute(select_sql)
                rows = cursor.fetchall()
                if not rows:
                    pbar.close()
                    print("No more rows to process.")
                    break

                for row in rows:
                    # Assuming the cursor returns rows as tuples: (id, md_content)
                    row_id = row[0]
                    content = row[1]
                    cleaned_content = clean_text(content) if content else None
                    
                    cursor.execute(update_sql, (cleaned_content, row_id))
                    # Update progress bar by one for each row processed.
                    pbar.update(1)

                # Commit after each batch.
                    conn.commit()

    except Exception as e:
        print("An error occurred:", e)
        conn.rollback()
    finally:
        conn.close()

In [7]:
process_rows()

Processing rows:   9%|▉         | 3/32 [00:02<00:19,  1.47row/s]


saved 2251799813690948

saved 1125899906844708


Processing rows:  12%|█▎        | 4/32 [00:02<00:12,  2.16row/s]


saved 2251799813687692

saved 2251799813687760


Processing rows:  22%|██▏       | 7/32 [00:03<00:05,  4.42row/s]

...

Processing rows: 53row [00:11,  7.56row/s]


saved 2251799813687508

saved 2251799813686084


Processing rows: 55row [00:12,  7.64row/s]


saved 1125899906844796

saved 1125899906845887


KeyboardInterrupt: 